# KM3NeT Masterclass Analysis

Upload the calibration csv file, then click **Run analysis** to execute `analysis.py`.


In [ ]:
%matplotlib inline

from pathlib import Path

import ipywidgets as widgets
from IPython.display import display, clear_output

from analysis import run_analysis

# ---- Paths configuration ----
INPUT_CSV = "sky_map.csv"   # e.g. "data/sky_map.csv"
OUTPUT_CSV = "scrambled_events.csv"
CALIBRATION_DEST = Path("config") / "calibration_nearly_perfect.csv"

In [ ]:
# ---- Widgets ----

upload = widgets.FileUpload(
    accept=".csv",
    multiple=False
)

run_button = widgets.Button(
    description="Run analysis",
    button_style="primary",
    tooltip="Run analysis.py with the uploaded calibration file"
)

# Progress bar (0..100)
progress = widgets.IntProgress(
    value=0,
    min=0,
    max=100,
    description="0%",
    bar_style="",
    orientation="horizontal"
)

output_area = widgets.Output()


def on_run_clicked(btn):
    with output_area:
        clear_output()

        # Reset progress
        progress.value = 0
        progress.description = "0%"
        progress.bar_style = ""

        if not upload.value:
            print("⚠ Please upload a calibration CSV file first.")
            return

        # Handle both ipywidgets 7 (dict) and 8+ (tuple) formats
        val = upload.value
        if isinstance(val, dict):
            file_info = next(iter(val.values()))
            # Some widget versions store the name differently
            filename = file_info.get("metadata", {}).get("name", file_info.get("name", "uploaded.csv"))
            content = file_info["content"]
        else:
            file_info = val[0]
            filename = getattr(file_info, "name", "uploaded.csv")
            content = getattr(file_info, "content", b"")

        # Save uploaded file to the canonical path expected by the repo
        CALIBRATION_DEST.parent.mkdir(parents=True, exist_ok=True)
        CALIBRATION_DEST.write_bytes(content)

        # Minimal output text (as requested previously)
        print(f"{filename} file uploaded")

        # Progress callback used by analysis.py
        def progress_cb(pct: int):
            pct = int(max(0, min(100, pct)))
            progress.value = pct
            progress.description = f"{pct}%"
            if pct >= 100:
                progress.bar_style = "success"

        try:
            run_analysis(
                infile=INPUT_CSV,
                outfile=OUTPUT_CSV,
                calibration_file=str(CALIBRATION_DEST),
                progress_callback=progress_cb,
                verbose=False
            )
        except Exception as e:
            progress.bar_style = "danger"
            print("❌ Error while running analysis:")
            print(e)
            return


run_button.on_click(on_run_clicked)

ui = widgets.VBox([
    widgets.HTML("<h2>KM3NeT Masterclass Calibration Viewer</h2>"),
    upload,
    run_button,
    progress,
    output_area
])

display(ui)